In [173]:
from math import exp
from random import seed
from random import random
import numpy as np
import matplotlib.pyplot as plt

# Initialize a network #w
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)] 
    network.append(hidden_layer) 
    output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer) 
    return network


def activate(weights, inputs):
    z = weights[-1] #bias is the last layer
    for i in range(len(weights)-1):
        z += weights[i] * inputs[i]
    return z

# Sigmoid neuron activation
def sigmoid(z):
    return 1.0 / (1.0 + exp(-z))

# Forward propagate input to a network output
def forward_propagate(network, sample):
    inputs = sample
    for layer in network:
        new_inputs = []
        for neuron in layer:
            z = activate(neuron['weights'], inputs)
            neuron['output'] = sigmoid(z)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs #We work through each layer of 

In [174]:
#backward propagation

In [175]:
def sigmoid_derivative(output): #g'(z)
    return output*(1.0 - output)

In [176]:
def back_propagate_error(network, y):
    for i in reversed(range(len(network))):
        layer=network[i]
        errors=list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error=0.0
                for neuron in network[i + 1]:
                    error=error + neuron['weights'][j] * neuron['delta']
                    errors.append(error)
        else:
            for j in range(len(layer)):
                neuron=layer[j]
                errors.append(y[j] - neuron['output'])
        for j in range(len(layer)):
            neuron=layer[j]
            neuron['delta']=errors[j]*sigmoid_derivative(neuron['output'])

In [177]:
def update_weights(network, sample, alpha):
    for i in range(len(network)):
        inputs=sample[:-1]
        if i != 0:
            inputs=[neuron['output'] for neuron in network[i - 1]]
            for neuron in network[i]:
                for j in range(len(inputs)):
                    neuron['weights'][j] = neuron['weights'][j] + alpha * neuron['delta'] * inputs[j]
                neuron['weights'][-1] = neuron['weights'][-1] + alpha * neuron['delta']

In [178]:
def train_network(network, train, alpha, epochs, n_outputs):
    error_list = []
    for epoch in range(epochs):
        sum_errors=0
        for sample in train:
            outputs=forward_propagate(network, sample)
            y=[0 for i in range(n_outputs)] 
            y[sample[-1]] = 1
            sum_errors = sum_errors + sum([(y[i] - outputs[i]) ** 2 for i in range(len(y))])
            back_propagate_error(network, y)
            update_weights(network, sample, alpha)
#         print('epoch=%d, lrate=%.3f, error = %.3f'%(epoch, alpha, sum_errors))
        error_list.append(sum_errors)
    return error_list


In [179]:
import pandas as pd

dataset=pd.read_csv('../dataset/LogisticRegressiondata.csv')
dataset

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
0,68.95,35,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0,Tunisia,2016-03-27 00:53:11,0
1,80.23,31,68441.85,193.77,Monitored national standardization,West Jodi,1,Nauru,2016-04-04 01:39:02,0
2,69.47,26,59785.94,236.50,Organic bottom-line service-desk,Davidton,0,San Marino,2016-03-13 20:35:42,0
3,74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,1,Italy,2016-01-10 02:31:19,0
4,68.37,35,73889.99,225.58,Robust logistical utilization,South Manuel,0,Iceland,2016-06-03 03:36:18,0
...,...,...,...,...,...,...,...,...,...,...
995,72.97,30,71384.57,208.58,Fundamental modular algorithm,Duffystad,1,Lebanon,2016-02-11 21:49:00,1
996,51.30,45,67782.17,134.42,Grass-roots cohesive monitoring,New Darlene,1,Bosnia and Herzegovina,2016-04-22 02:07:01,1
997,51.63,51,42415.72,120.37,Expanded intangible solution,South Jessica,1,Mongolia,2016-02-01 17:24:57,1
998,55.55,19,41920.79,187.95,Proactive bandwidth-monitored policy,West Steven,0,Guatemala,2016-03-24 02:35:54,0


In [180]:
dataset = dataset.drop(['Ad Topic Line', 'Timestamp', 'City', 'Country'], axis = 1).values

In [181]:
label = dataset[:,-1]
np.shape(label)

(1000,)

In [182]:
dataset = dataset[:,:-1]
dataset = (dataset - dataset.min()) / (dataset.max() - dataset.min())
dataset = np.insert(dataset, len(dataset[0]), label, axis = 1)
print(dataset)

[[8.67461452e-04 4.40335762e-04 7.77933643e-01 3.22187387e-03
  0.00000000e+00 0.00000000e+00]
 [1.00937538e-03 3.90011675e-04 8.61068406e-01 2.43782459e-03
  1.25810218e-05 0.00000000e+00]
 [8.74003583e-04 3.27106566e-04 7.52168213e-01 2.97541165e-03
  0.00000000e+00 0.00000000e+00]
 ...
 [6.49558155e-04 6.41632111e-04 5.33633097e-01 1.51437759e-03
  1.25810218e-05 1.00000000e+00]
 [6.98875760e-04 2.39039414e-04 5.27406372e-01 2.36460304e-03
  0.00000000e+00 0.00000000e+00]
 [5.66271790e-04 3.27106566e-04 3.75868091e-01 2.24382523e-03
  0.00000000e+00 1.00000000e+00]]


In [183]:
dataset = dataset.tolist()

In [184]:
np.shape(dataset)
print(len(dataset[1]))

6


In [185]:
for row in dataset:
    row[-1] = int(row[-1])

In [186]:
from random import seed
seed(1)
n_inputs=len(dataset[0]) - 1
n_hidden=2
n_outputs=len(set([sample[-1] for sample in dataset]))
network=initialize_network(n_inputs, 2, n_outputs)
alpha=0.03
epochs=10000
initialize_network(n_inputs, n_hidden, n_outputs)

[[{'weights': [0.9014274576114836,
    0.030589983033553536,
    0.0254458609934608,
    0.5414124727934966,
    0.9391491627785106,
    0.38120423768821243]},
  {'weights': [0.21659939713061338,
    0.4221165755827173,
    0.029040787574867943,
    0.22169166627303505,
    0.43788759365057206,
    0.49581224138185065]}],
 [{'weights': [0.23308445025757263, 0.2308665415409843, 0.2187810373376886]},
  {'weights': [0.4596034657377336,
    0.28978161459048557,
    0.021489705265908876]}]]

In [187]:
print(type(dataset))
train_data = dataset[:][:800]
test_data = dataset[:][801:]

<class 'list'>


In [ ]:
errors = train_network(network, train_data, alpha, epochs, n_outputs)
# for layer in network:
#     print(layer)

In [ ]:
network

In [ ]:
n_inputs

In [ ]:
n_outputs

In [ ]:
output = forward_propagate(network, test_data[0])

In [ ]:
def predict(network, sample):
    outputs=forward_propagate(network, sample)
    return outputs.index(max(outputs))

In [ ]:
for sample in test_data:
    prediction=predict(network, sample)
#     print(sample[-1], prediction)

In [ ]:
plt.plot(range(10000), errors)